# build_compilation notebook

This notebook iterates through each paleomagnetic record (datasheet) of the vgp database, extracts data which meet user-specified criteria, and appends them to a new dataframe for later processing (to generate an APWP).

In [1]:
import os
import re
import numpy as np
import pandas as pd
from pmagpy import ipmag, pmag
from scripts.auxiliar import get_files_in_directory, spherical2cartesian, GCD_cartesian
pd.set_option('display.max_columns', None)

Set the directory from which we will pull the datasheets. These should be available as .csv files (one for each record according to the vgp template).

In [2]:
data_path_vgp = os.getcwd() + '/vgp_database'
files_names = get_files_in_directory(data_path_vgp)
csv_file_names = [os.path.splitext(os.path.basename(open(file,'r').name))[0] for file in files_names if file.endswith('.csv')] #consider just *csv files
paths = [file for file in files_names if file.endswith('.csv')] 
files = pd.DataFrame({'path': paths, 'name': csv_file_names})

## Data selection criteria
Specify the specific inclusion criteria to be used in the data-selection. If author_selection is set=1, all other criteria will be ignored. Setting values other than 'None' for the remaining criteria allow entries that were rejected by the original authors to be retrieved. This also allows selection criteria between studies to be easily homogenized.

In [3]:
incl_criteria = {
    'author_selection': 1,     # 1 (yes) or 0 (no); if 1, all other criteria will be ignored
    'undemagnetized': None,    # None (defaults to author selection) or 'y'  
    'sample_count': None,      # None (defaults to author selection) or int: cutoff n (≥ x)
    'alpha_95': None,          # None (defaults to author selection) or float: cutoff A95 (≤ x degrees)
    'overprints': None,        # None (defaults to author selection) or 'y'  
    'remagnetizations': None,  # None (defaults to author selection) or 'y'
    'uncertain_struct': None,  # None (defaults to author selection) or 'y'
    'rotated': None,           # None (defaults to author selection) or 'y'
    'shallowed': None,         # None (defaults to author selection) or 'y' [***can also implement cutoff f-value here if desired***]
    'anomalous_dir': None,     # None (defaults to author selection) or float: cutoff distance (in degrees) between vgp and mean (≤ x degrees)
    'uncertain_age': None,     # None (defaults to author selection) or float: cutoff age resolution (in Myr) between min and max (≤ x Myr)
    'distinct_age': None,      # None (defaults to author selection) or 'y'
    'sub-time_units': None,    # None (defaults to author selection) or 'y'
    'rock_type': None,         # None (defaults to author selection) or string: 'all' or 'igneous' or 'sedimentary'
    'otherwise_rej': None,     # None (defaults to author selection) or 'y'
    }

Check the selection criteria and convert to numeric codes as used in the vgp database

In [4]:
criteria_codes = []
if incl_criteria['author_selection'] == 1: pass #ignore all other criteria if original selection is to be used
elif incl_criteria['author_selection'] == 0:
    if incl_criteria['undemagnetized'] == 'y': criteria_codes.append(1)
    if incl_criteria['sample_count'] == type(int): criteria_codes.append(2)
    if incl_criteria['alpha_95'] == type(float) or incl_criteria['alpha_95'] == type(int): criteria_codes.append(3)
    if incl_criteria['overprints'] == 'y': criteria_codes.append(4)
    if incl_criteria['remagnetizations'] == 'y': criteria_codes.append(5)
    if incl_criteria['uncertain_struct'] == 'y': criteria_codes.append(6)
    if incl_criteria['rotated'] == 'y': criteria_codes.append(7)
    if incl_criteria['shallowed'] == 'y': criteria_codes.append(8)
    if incl_criteria['anomalous_dir'] == type(float) or incl_criteria['anomalous_dir'] == type(int): criteria_codes.append(9)
    if incl_criteria['uncertain_age'] == type(float) or incl_criteria['uncertain_age'] == type(int): criteria_codes.append(10)    
    if incl_criteria['distinct_age'] == 'y': criteria_codes.append(11)
    if incl_criteria['sub-time_units'] == 'y': criteria_codes.append(12)
    if incl_criteria['rock_type'] == 'all' or incl_criteria['rock_type'] == 'igneous' or incl_criteria['rock_type'] == 'sedimentary': criteria_codes.append(13)
    if incl_criteria['otherwise_rej'] == 'y': criteria_codes.append(14)
else:
    print ('invalid inclusion criterion selected for author_selection')
    
print ('the numeric codes selected: ', criteria_codes)

the numeric codes selected:  []


## Initialize objects
We seek to append all the data extracted from each datasheet into one place, we therefore initialize a master dataframe for vgps. We will also recalculate the study-level paleopoles based on our revised vgp selections, and so we also initialize a master dataframe for these recalculated poles.

In [5]:
df_vgps_master = pd.DataFrame()
df_recalc_pps_master = pd.DataFrame()

Several elements of the vgp database involve references between sites of a given datasheet: namely, stratigraphic ordering and common (redundant) instantaneous records of the field (e.g. 2 or more sites from the same cooling unit). This referencing is achieved by way of simple numeric codes which are repeated across datasheets. In order to preserve these references when we merge datasheets, we need to ensure these codes are made unique. This is easily achieved with use of counters, which we initialize at 0.

In [6]:
strat_group_counter = 0
time_unit_counter = 0 

## Example
In order to demonstrate the processes executed below, we select an arbitrary single datasheet to process, before executing the same process on the entire database below. We first select a datasheet from among those in the directory.

In [7]:
files[['name']]

,name
0,test
1,test2


In [8]:
file_idx = 1

### Split study- and site-level data
Each datasheet contains both study-level poles and site-level vgps. We split these data and assign them to separate dataframes. We can also cast types for specific columns to avoid problems later.

In [9]:
def split_datasheet (files, file_idx): 
    df = pd.read_csv(files['path'][file_idx], skip_blank_lines = False, encoding = "ISO-8859-1") #, skip_blank_lines=True
    df_list = np.split(df, df[df.isnull().all(1)].index)
    df_poles = df_list[0]
    df_vgps = df_list[1].dropna(how='all')
    df_vgps =df_vgps.rename(columns = df_vgps.iloc[0]).drop(df_vgps.index[0]) # assign the first row as columns for the df_vgps

    #cast columns
    df_poles = df_poles.astype({'pole': int, 'N': int,
                              "slat":float, "slon":float, "dec":float, "inc":float,
                              "Plat":float, "Plon":float})
    df_vgps = df_vgps.astype({'in_study_pole': int, 'strat_group': int, 'n': int, 'k': float,
                              "slat":float, "slon":float, "dec":float, "inc":float, 'alpha95': float, 'time_unit': str,
                             'min_age':float, 'max_age': float, 'mean_age':float,
                              "VGP_lat":float, "VGP_lon":float})
    return (df_poles, df_vgps)

In [10]:
df_poles, df_vgps = split_datasheet(files, file_idx)

Have a look at these dataframes

In [11]:
df_poles.head()

,pole,name,slat,slon,N,dec,inc,k,alpha95,f_corr,Plat,Plon,K,A95,dp,dm,mean_age,2sig_mean,min_age,2sig_min,max_age,2sig_max,error_dist,lithology_1,lithology_2,R1,R2,R3,R4,R5,R6,R7,pmag_ref,age_ref,pmag_comments,age_comments
0,1,"Quaternary age rocks, Western Central TMVB",NaN,NaN,10,2.9,37.5,38,7.9,NaN,86.7,314.0,43,7.5,NaN,NaN,NaN,NaN,0,NaN,2.58,NaN,uniform,igneous; volcanic,mafic to intermediate lavas,1,1,1,0,0.5,1,1,Ruiz-Martínez et al. (2010),GTS2020,no field stability tests; structural coherence...,NaN
1,2,"late Miocene-Pliocene age rocks, Western Centr...",NaN,NaN,33,0.6,38.2,22,5.4,NaN,88.0,265.5,26,5,NaN,NaN,NaN,NaN,2.58,NaN,11.2,NaN,uniform,igneous; volcanic,mafic to intermediate lavas,1,1,1,0,0.5,1,1,Ruiz-Martínez et al. (2010),GTS2020,no field stability tests; structural coherence...,min age is Pliocene-Quaternary boundary; max a...


In [12]:
df_vgps.head()

,name,fm./loc.,slat,slon,n,dec,inc,k,alpha95,f_corr,VGP_lat,VGP_lon,K,A95,dp,dm,mean_age,2sig_mean,min_age,2sig_min,max_age,2sig_max,error_dist,lithology_1,lithology_2,polarity,strat_group,ordering,level,time_unit,in_study_pole,rej_crit,pmag_ref,age_ref,pmag_comments,age_comments
4,mwnHIG,Western sector of TMVB,20.79,-105.48,11,343.6,24.6,37.0,6.9,NaN,72.4,140.4,NaN,NaN,NaN,NaN,10.20,0.8,NaN,NaN,NaN,NaN,normal,igneous; volcanic,mafic to intermediate lavas,NaN,0,NaN,NaN,0,2,NaN,Ruiz-Martínez et al. (2010),Ruiz-Martínez et al. (2010),NaN,NaN
5,mwrSF,Western sector of TMVB,20.89,-105.41,11,190.8,-45.3,2.0,30.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.10,0.2,NaN,NaN,NaN,NaN,normal,igneous; volcanic,mafic to intermediate lavas,NaN,0,NaN,NaN,0,0,3,Ruiz-Martínez et al. (2010),Ruiz-Martínez et al. (2010),scattered directions,NaN
6,pwnPLA,Western sector of TMVB,21.35,-105.24,8,351.1,31.3,60.0,6.4,NaN,80.5,138.4,NaN,NaN,NaN,NaN,NaN,NaN,2.58,NaN,3.5,NaN,uniform,igneous; volcanic,mafic to intermediate lavas,NaN,0,NaN,NaN,0,2,NaN,Ruiz-Martínez et al. (2010),Ruiz-Martínez et al. (2010),NaN,NaN
7,pwrLIB,Western sector of TMVB,21.58,-105.19,10,166.5,-41.1,492.0,2.2,NaN,77.4,176.4,NaN,NaN,NaN,NaN,NaN,NaN,2.58,NaN,3.5,NaN,uniform,igneous; volcanic,mafic to intermediate lavas,NaN,0,NaN,NaN,0,2,NaN,Ruiz-Martínez et al. (2010),Ruiz-Martínez et al. (2010),NaN,NaN
8,pwrJOL,Western sector of TMVB,21.40,-105.18,7,8.4,49.1,78.0,6.9,NaN,78.6,294.4,NaN,NaN,NaN,NaN,3.36,0.17,NaN,NaN,NaN,NaN,normal,igneous; volcanic,mafic to intermediate lavas,NaN,0,NaN,NaN,2,2,NaN,Ruiz-Martínez et al. (2010),Ruiz-Martínez et al. (2010),NaN,NaN


## Fill-in missing data
The datasheets in the database are mostly only populated with data reported in the original publication, which means that some series will be empty. In some cases, we can estimate values for these missing series based on other reported data (e.g. computing a vgp from reported dec/inc and slat/slon values). In the following we seek to populate the specific series that we may need to utilize later.

### Check / fill-in any missing vgps.
(or otherwise drop entry)

In [13]:
def get_vgps (df, file_idx): #seeks to fill in missing vgp entries in dataframe
    
    # first identify any entries missing vgp information
    df['VGP_exists'] = df.apply(lambda row: True if not (np.isnan(row.VGP_lat) or np.isnan(row.VGP_lon)) else False, axis=1)
    df_missing_vgps = df[df['VGP_exists'] == False]
    
    if df_missing_vgps.empty:
        print ('no missing vgp information')
    
    else:
        # now check that those which are missing vgp data have sufficient information to calculate it (dec/inc + site data)
        df_missing_vgps['sufficient'] = df_missing_vgps.apply(lambda row: True if not (np.isnan(row.slat) or np.isnan(row.slon) or np.isnan(row.dec) or np.isnan(row.inc)) \
                                                    else False, axis=1)

        # report any sites where critical information is lacking 
        if not df_missing_vgps['sufficient'].all():
            missing_idx = df_missing_vgps.index[df_missing_vgps['sufficient'] == False].tolist()
            for i in missing_idx:
                site = df['name'][i]
                print (f'Missing slat/slon and/or dec/inc at site {site} in file index {file_idx} where no vgp is reported; cannot calculate vgp -- dropping entry') 

            # drop entries with no vgp
            df.drop(labels=missing_idx, inplace=True)
                
        # calculate vgps. This adds columns: 'paleolatitude', 'vgp_lat', 'vgp_lon', 'vgp_lat_rev' and 'vgp_lon_rev'
        df_get_vgps = df_missing_vgps[df_missing_vgps['sufficient'] == True]
        ipmag.vgp_calc(df_get_vgps, site_lon='slon', site_lat='slat', dec_tc='dec', inc_tc='inc')

        # assign calculated vgps to original dataframe
        df.VGP_lat.fillna(df_get_vgps.vgp_lat, inplace=True)
        df.VGP_lon.fillna(df_get_vgps.vgp_lon, inplace=True)
    
    df.drop(['VGP_exists'], axis=1, inplace=True)
    
    return df

In [14]:
df_vgps = get_vgps(df_vgps, file_idx)

Missing slat/slon and/or dec/inc at site pwrIXT in file index 1 where no vgp is reported; cannot calculate vgp -- dropping entry


### Check / fill-in any missing site coordinate information.
(or otherwise drop entry)

In [15]:
def get_sites (df, file_idx): #seeks to fill in missing site coordinates in dataframe
    
    # first identify any entries missing site information
    df['site_exists'] = df.apply(lambda row: True if not (np.isnan(row.slat) or np.isnan(row.slon)) else False, axis=1)
    df_missing_sites = df[df['site_exists'] == False]
    
    if df_missing_sites.empty:
        print ('no missing site information')
    
    else:
        # now check that those which are missing site data have sufficient information to calculate it (dec/inc + vgp coordinates)
        df_missing_sites['sufficient'] = df_missing_sites.apply(lambda row: True if not (np.isnan(row.slat) or np.isnan(row.slon) or np.isnan(row.dec) or np.isnan(row.inc)) \
                                                    else False, axis=1)

        # report any sites where critical information is lacking 
        if not df_missing_sites['sufficient'].all():
            missing_idx = df_missing_sites.index[df_missing_sites['sufficient'] == False].tolist()
            for i in missing_idx:
                location = df['name'][i]
                print (f'Missing dec/inc and/or vgp at site {location} in file index {file_idx} where no site coordinates are reported; cannot calculate site coordinates -- dropping entry')

        # calculate site coordinates. ### *** THIS FUNCTION REMAINS TO BE BUILT *** ###
        df_get_site = df_missing_sites[df_missing_sites['sufficient'] == True]
        ### FUNCTION TO BE BUILT

        # assign calculated site coordinates to original dataframe
        #df.slat.fillna(df_get_site.site_lat, inplace=True)
        #df.slon.fillna(df_get_site.site_lon, inplace=True)

        # drop entries with no vgp and the added column
        df.drop(labels=missing_idx, inplace=True)
    
    df.drop(['site_exists'], axis=1, inplace=True)
    
    return df

In [16]:
df_vgps = get_sites(df_vgps, file_idx)

no missing site information


### Cross-check vgps against dec/inc and slat/slon
Now that we have ensured all the key data are filled in, check that the vgps are self-consistent with the dec/inc and slat/slon data.
Where poles appear to have been inverted, flip back to the correct polarity. Flag any otherwise spurious vgps to be checked against original reference.

In [17]:
def check_vgps (df, file_idx): #check vgps against dec/inc and slat/slon data
    
    # compute vgp from dec/inc & slat/slon
    ipmag.vgp_calc(df, site_lon='slon', site_lat='slat', dec_tc='dec', inc_tc='inc')
    
    # measure distance between recalculated vgp and listed vgp
    df['GCD'] = df.apply(lambda row: pmag.angle([row.VGP_lon, row.VGP_lat], [row.vgp_lon, row.vgp_lat]), axis=1)
    
    # if angle is greater than 178 degrees, assume it was inverted by original authors and re-invert
    invert_idx = df.index[df['GCD'] > 178.0].tolist()
    for i in invert_idx:
        location = df['name'][i]
        print (f'vgp from site {location} in file index {file_idx} appears to be inverted. Flipping back.')
    
    df['VGP_lat'] = np.where(df['GCD'] > 178., -df['VGP_lat'], df['VGP_lat'])
    df['VGP_lon'] = np.where(df['GCD'] > 178., (df['VGP_lon']-180.) % 360., df['VGP_lon'])
    
    # if any angle is between 2 and 178 degrees, flag it as spurious
    spurious_idx = df.index[(df['GCD'] > 2.0) & (df['GCD'] < 178.0)].tolist()
    for i in spurious_idx:
        location = df['name'][i]
        angle = int(df['GCD'][i])
        print (f'***SPURIOUS*** vgp from site {location} in file index {file_idx}; reported pole differs from re-calculated by {angle} degrees. CHECK against original reference')
        
    # drop added columnS
    df.drop(['GCD', 'paleolatitude', 'vgp_lat', 'vgp_lon', 'vgp_lat_rev', 'vgp_lon_rev'], axis=1, inplace=True)
    
    return df

In [18]:
df_vgps = check_vgps (df_vgps, file_idx)

vgp from site pwrLIB in file index 1 appears to be inverted. Flipping back.
vgp from site pwnPAL in file index 1 appears to be inverted. Flipping back.
vgp from site pwrJAL in file index 1 appears to be inverted. Flipping back.
vgp from site pwrSJG in file index 1 appears to be inverted. Flipping back.
vgp from site pwrFER in file index 1 appears to be inverted. Flipping back.
vgp from site mcrARE in file index 1 appears to be inverted. Flipping back.
vgp from site pcrCHA in file index 1 appears to be inverted. Flipping back.
vgp from site mcrFIN in file index 1 appears to be inverted. Flipping back.
vgp from site pcrTRO in file index 1 appears to be inverted. Flipping back.
vgp from site pcrOCO in file index 1 appears to be inverted. Flipping back.
vgp from site qcrCG in file index 1 appears to be inverted. Flipping back.
vgp from site pcrPEN in file index 1 appears to be inverted. Flipping back.
vgp from site qcrEST in file index 1 appears to be inverted. Flipping back.
vgp from site

### Establish common polarity convention
Add an extra column where all vgps are cast into reverse polarity to make common calculations easier.

In [19]:
def get_common_polarity (df): # cast all vgps into the southern hemisphere
    
    # *** NOTE: This function should be generalized so that we don't invert low-latitude reverse polarity poles the wrong way!
    # this will be increasingly more important as we go further backward in time, as we cannot rely on the assumption that reverse
    # polarity poles should be in southern hemisphere

    df['VGP_lat_Rpol'] = np.where(df['VGP_lat'] > 0, -df['VGP_lat'], df['VGP_lat'])
    df['VGP_lon_Rpol'] = np.where(df['VGP_lat'] > 0, (df['VGP_lon'] - 180.) % 360., df['VGP_lon'])
    
    return df

In [20]:
df_vgps = get_common_polarity(df_vgps)

### Check / fill-in any missing alpha 95's.
(or otherwise set to 999)

In [21]:
def get_alpha95s (df, file_idx): #seeks to fill in missing alpha95s in dataframe
    
    # first identify any entries missing alpha95 information
    df['a95_exists'] = df.apply(lambda row: True if not np.isnan(row.alpha95) else False, axis=1)
    df_missing_a95s = df[df['a95_exists'] == False]
    
    if df_missing_a95s.empty:
        print ('no missing alpha95 information')

    else:
        # now check that those which are missing alpha95 data have sufficient information to calculate it (n & k)
        df_missing_a95s['sufficient'] = df_missing_a95s.apply(lambda row: True if not (np.isnan(row.n) or np.isnan(row.k)) \
                                                    else False, axis=1)

        # report any sites where critical information is lacking 
        if not df_missing_a95s['sufficient'].all():
            missing_idx = df_missing_a95s.index[df_missing_a95s['sufficient'] == False].tolist()
            for i in missing_idx:
                location = df['name'][i]
                print (f'Missing n and/or k at site {location} in file index {file_idx} where no alpha95 is reported; cannot calculate alpha95 -- setting to 999')

        # calculate site coordinates. ### *** THIS FUNCTION REMAINS TO BE BUILT *** ###
        df_get_a95s = df_missing_a95s[df_missing_a95s['sufficient'] == True]
        df_get_a95s['a95'] = df_get_a95s.apply(lambda row: 140.0/np.sqrt(row.n * row.k), axis=1)

        # assign calculated a95s to original dataframe, set those which could not be calculated to 999
        df.alpha95.fillna(df_get_a95s.a95, inplace=True)
        
        # set those which could not be calculated to 999, and drop added column
        df.alpha95.fillna(value=999)
    
    df.drop(['a95_exists'], axis=1, inplace=True)
    
    return df

In [22]:
df_vgps = get_alpha95s(df_vgps, file_idx)

no missing alpha95 information


#### TO DO: fill in min and max ages for normally distributed ages
<em> OR SHOULD WE JUST SIMPLIFY DATASHEETS TO ONLY INCLUDE MEAN AGE, MIN_AGE, MAX_AGE & ERROR_TYPE? </em>  
... for time being have just made function to ensure mean ages are filled in ...

In [23]:
def get_ages (df):
    
    df['mean_age'] = df.apply(lambda row: (row.min_age + row.max_age)/2.0 if np.isnan(row.mean_age) else row.mean_age, axis=1)
   
    return df

In [24]:
df_vgps = get_ages(df_vgps)

## Assign unique IDs to strat_groups and time_units
Change the numeric codes assigned to strat_group and time_unit entries according to counters so that they are uniquely identifiable after merger into the central dataframe.

In [25]:
def assign_uniq_ids (df, strat_group_counter, time_unit_counter):  #assigns new codes to local strat_groups and time_units to make unique across entire database
    
    # assign new codes to strat_groups
    df['strat_group'] = df.apply(lambda row: row.strat_group + strat_group_counter if not row.strat_group == 0 else row.strat_group, axis=1)

    #update strat_group_counter with new max value from local list
    strat_group_counter = df['strat_group'].max()

    # update time_unit identifiers (note that some entries have an 'M' prefix that designates them as a local mean)
    df.time_unit.fillna(value='0')
    df.time_unit = df.time_unit.astype('str') # ensure that time_unit entries are strings
    df['time_unit'] = df.apply(lambda row: ' '.join(re.findall("[a-zA-Z]+", row.time_unit)) + str(int(''.join(filter(str.isdigit, row.time_unit))) \
                                                    + time_unit_counter) if not row.time_unit == '0' else row.time_unit, axis = 1)
    
    #update time_unit_counter with new max value
    time_unit_counter = pd.to_numeric(df['time_unit'], 'coerce').max()

    return (df, strat_group_counter, time_unit_counter)

In [26]:
df_vgps, strat_group_counter, time_unit_counter = assign_uniq_ids(df_vgps, strat_group_counter, time_unit_counter)

Because we will loop over the entire database below (including this datasheet), we will here reset the counter back to 0.

In [27]:
strat_group_counter = 0
time_unit_counter = 0

## Filter data
Now we evaluate the entries against the specified inclusion criteria. We start with an 'initial' filter, which removes any entries that don't have the right basic inclusion codes and those which fail any specified n, alpha95, age uncertainty and/or rock type criteria.

In [28]:
def init_filter (df, incl_criteria, criteria_codes): #flag the subset of entries which fulfill the specified inclusion criteria
    
    df['rej_crit'] = df['rej_crit'].fillna(0) # replace NaNs in this column with 0's
    
    # make a new temporary series that first flags those entries which do / don't pass the basic inclusion criteria according to rej_crit codes
    df['keep'] = df.apply(lambda row: True if row.in_study_pole != 0 or all(crit in criteria_codes for crit in [int(i) for i in str(row.rej_crit).split(',')]) \
                                else False, axis=1)
    
    # reject any entries with too small sample count
    if 2 in criteria_codes:
        df['keep'] = df.apply(lambda row: False if row.n < incl_criteria['sample_count'] else row.keep, axis=1)

    # reject any entries with too large alpha 95
    if 3 in criteria_codes:  
        df['keep'] = df.apply(lambda row: False if row.alpha95 > incl_criteria['alpha_95'] else row.keep, axis=1)

    # reject vgps with too large age uncertainty (as determined by diff b/w min and max)
    if 10 in criteria_codes:
        df['keep'] = df.apply(lambda row: False if (row.max_age - row.min_age) > incl_criteria['uncertain_age'] else row.keep, axis=1)

    # reject vgps with wrong rock type
    if 13 in criteria_codes: 
        if incl_criteria['rock_type'] == 'all': pass
        else: df['keep'] = df.apply(lambda row: False if row.rock_type != incl_criteria['rock_type'] else row.keep, axis=1)
            
    df.drop(df[df.keep == False].index, inplace=True)
    
    return df

In [29]:
df_vgps = init_filter(df_vgps, incl_criteria, criteria_codes)

We still need to find and reject any anomalous vgps, if any such criteria were specified above. However, in order to evaluate this, we need a provisional paleopole. Before we compute that we need to remove any vgps with distinct ages (since they shouldn't contribute to the paleopole calculation). These temporally distinctive entries can be sent to the master vgp dataframe and removed from our local selection.

In [30]:
def strip_age_distinct (df, df_master, criteria_codes):

    # check if any entries with distinct age and flag with a NaN in the 'keep' column
    if 11 in criteria_codes:
        df['keep'] = df.apply(lambda row: np.nan if (row.keep == True and 11 in [int(i) for i in str(row.rej_crit).split(',')]) else row.keep, axis=1) 

        #pass these distinct age vgps to the master vgp dataframe
        df_distinct = df[df['keep'] == np.nan]
        df_master = pd.concat([df_master, df_distinct], axis=0)

        #drop the distinct vgps from the selected list
        df.drop(df_distinct.index, axis=0, inplace=True)
    
    return (df, df_master)

In [31]:
df_vgps, df_vgps_master = strip_age_distinct(df_vgps, df_vgps_master, criteria_codes)

## Compute provisional paleopole
Now we can compute a provisional paleopole and check for anomalous vgps (according to the definition of 'anomalous' as specified in the inclusion criteria). To rigorously compute a pole presents something of a headache because we should first pre-average any common-time units, but some of these could potentially include anomalous directions that would susequently be dismissed, etc. Here we adopt a simpler approach: defaulting to the subset of selected data which the original authors also retained.

In [32]:
def strip_anomalous (df, criteria_codes):

    if 9 in criteria_codes: 
        
        #isolate the entries to be used for provisional paleopole calculation
        df_prov = df[(df['in_study_pole'] != 0) & (df['keep'] == True)]

        #calculate provisional paleopole
        ppole = ipmag.fisher_mean(dec = df_prov['VGP_lon_Rpol'].tolist(), inc = df_prov['VGP_lat_Rpol'].tolist())
        print ('Provisional pole: ', ppole)

        #identify anomalous vgps according to the specification above
        df['keep'] = df.apply(lambda row: False if (pmag.angle([row.VGP_lon_Rpol, row.VGP_lat_Rpol], [ppole['dec'], ppole['inc']]) \
                                                              > incl_criteria['anomalous_dir']) else row.keep, axis=1)

        #drop anomalous entries
        df.drop(df[df.keep == False].index, inplace=True)

    return df

In [33]:
df_vgps = strip_anomalous(df_vgps, criteria_codes)

### Remove sparse time_units collections
After filtering there may be some common time_unit collections with too few entries to be useful to pass onward to the master dataframe. In these cases, we can adopt any existing reported mean and discard the individual entries. In other cases, where there exists a sufficient number of individual entries we can pass them on and omit the reported mean. For this we need to decide what is 'too few': we default to either the minimum number of samples (n) as specified above or <= 3 (whichever is higher), but this can otherwise be adjusted to any other arbitrary value larger than 3.

In [34]:
def strip_sparse_time_units (df, incl_criteria):

    # first specify an n-specific cutoff value to decide whether to recalculate mean or retain the reported one.
    min_site_count = 3
    if incl_criteria['sample_count'] != None and incl_criteria['sample_count'].isdigit():
        if incl_criteria['sampl_count'] > min_site_count: min_site_count = int(incl_criteria['sampl_count'])
            
    # now extract common time units from selected list and split into groups
    df['keep'] = df.apply(lambda row: False if row.time_unit != '0' else row.keep, axis=1)
    df_redundant = df[df['time_unit'] != '0']
    time_units = df_redundant.groupby(['time_unit'])
    
    # collect the means into a list to make them easy to find when needed
    means = [x for x in time_units.groups if x.isdigit() == False]

    # now check the number of sites for each given group
    for key, group in time_units:
        if key.isdigit():    # ignore means
            
            if len(group) > min_site_count:   # if number of sites is sufficient, keep all individual sites
                df.loc[group.index.tolist(), 'keep'] = True

            elif ('M'+str(key)) in means:     # if number of sites is too low and mean is reported, keep mean
                mean_ent = time_units.get_group('M'+str(key))
                df.loc[mean_ent.index.tolist(), 'keep'] = True
                df.loc[mean_ent.index.tolist(), 'time_unit'] = '0' # set time_unit to 0 as there is now only 1 entry

            else:      # if number of sites is too low and no mean is reported, use site with smaller alpha95 (or higher n)
                df.loc[group['alpha95'].idxmin(), 'keep'] = True
                ### alternatively: group['n'].idxmax()
                df.loc[group['alpha95'].idxmin(), 'time_unit'] = '0'  # set time_unit to 0 as there is now only 1 entry
                
    #drop discarded entries
    df.drop(df[df.keep == False].index, inplace=True)
    
    return df

In [35]:
df_vgps = strip_sparse_time_units(df_vgps, incl_criteria)

## Append filtered data to master dataframe
Pass the final selected entries to the master vgp dataframe.

In [36]:
df_vgps_master = pd.concat([df_vgps_master, df_vgps], axis=0)

## Final paleopole recalculation
Finally, we can recalculate the paleopole based on only the filtered set of vgp data. First we need to pre-average the common time_units.

In [37]:
def average_time_units (df): # pre-average the common time units
    
    df['keep'] = df.apply(lambda row: False if row.time_unit != '0' else row.keep, axis=1)
    df_redundant = df[df['time_unit'] != '0']
    time_units = df_redundant.groupby(['time_unit'])
    
    for key, group in time_units:
        mean_age = group['mean_age'].mean(axis=0) # get mean age from among time_unit sites *** NOTE THIS DOESN'T COLLECT / PASS ON UNCERTAINTIES ***
        
        vgp = ipmag.fisher_mean(dec = group['VGP_lon_Rpol'].tolist(), inc = group['VGP_lat_Rpol'].tolist()) # compute vgp from among time_units
        
        df.append({'VGP_lon': vgp['dec'], 'VGP_lon_Rpol': vgp['dec'], 'VGP_lat': vgp['inc'], 'VGP_lat_Rpol': vgp['inc'],
                   'A95': vgp['alpha95'], 'mean_age': mean_age, 'keep': True}, ignore_index=True)    # other entries could be added but aren't presently needed
        
    #drop discarded entries
    df.drop(df[df.keep == False].index, inplace=True)
    
    return df    

In [38]:
df_vgps = average_time_units(df_vgps)

Now compute the final paleopole, determine its corresponding age, and append it to the re-calculated paleopole dataframe.

In [39]:
def compute_pole (df):
    
    mean_age = df['mean_age'].mean(axis=0) # get mean pole age  *** NOTE THIS DOESN'T COLLECT / PASS ON UNCERTAINTIES ***
    
    paleopole = ipmag.fisher_mean(dec = df['VGP_lon_Rpol'].tolist(), inc = df['VGP_lat_Rpol'].tolist())
    
    print (mean_age, paleopole['dec'], paleopole['inc'])
    
# df_recalculated_poles_master = pd.concat([df_recalc_pps_master, recalc_pp], axis=0)

In [40]:
df_vgps = compute_pole(df_vgps)

4.115694444444444 82.75842518665542 -88.3440171117142


## Execute on the entire dataframe
Now that we have demonstrated the workflow on an example datasheet, we can execute it across the entire dataset by cycling through all the datasheets.

In [41]:
for i in files.index:   # cycle over each file in database
    
    df_poles, df_vgps = split_datasheet(files, i)
    df_vgps = get_vgps(df_vgps, i)
    df_vgps = get_sites(df_vgps, i)
    df_vgps = check_vgps (df_vgps, i)
    df_vgps = get_common_polarity(df_vgps)
    df_vgps = get_alpha95s(df_vgps, i)
    df_vgps = get_ages(df_vgps)
    df_vgps, strat_group_counter, time_unit_counter = assign_uniq_ids(df_vgps, strat_group_counter, time_unit_counter)
    df_vgps = init_filter(df_vgps, incl_criteria, criteria_codes)
    df_vgps, df_vgps_master = strip_age_distinct(df_vgps, df_vgps_master, criteria_codes)
    df_vgps = strip_anomalous(df_vgps, criteria_codes)
    df_vgps = strip_sparse_time_units(df_vgps, incl_criteria)
    df_vgps_master = pd.concat([df_vgps_master, df_vgps], axis=0)
    
    df_vgps = average_time_units(df_vgps)
    df_vgps = compute_pole(df_vgps)

no missing site information
vgp from site pwrLIB in file index 0 appears to be inverted. Flipping back.
vgp from site pwnPAL in file index 0 appears to be inverted. Flipping back.
vgp from site pwrJAL in file index 0 appears to be inverted. Flipping back.
vgp from site pwrSJG in file index 0 appears to be inverted. Flipping back.
vgp from site pwrFER in file index 0 appears to be inverted. Flipping back.
vgp from site mcrARE in file index 0 appears to be inverted. Flipping back.
vgp from site pcrCHA in file index 0 appears to be inverted. Flipping back.
vgp from site mcrFIN in file index 0 appears to be inverted. Flipping back.
vgp from site pcrTRO in file index 0 appears to be inverted. Flipping back.
vgp from site pcrOCO in file index 0 appears to be inverted. Flipping back.
vgp from site qcrCG in file index 0 appears to be inverted. Flipping back.
vgp from site pcrPEN in file index 0 appears to be inverted. Flipping back.
vgp from site qcrEST in file index 0 appears to be inverted. 

In [42]:
### Other things to do:
### - implement capacity to eject results from poles that are known to be regionally rotated (this requires additional fields being added to the database)